In [ ]:
!pip install pandas

In [ ]:
!pip install networkx

In [ ]:
!pip install joblib

In [ ]:
!pip install gensim

In [66]:
from network import CNetwork

In [67]:
import pandas as pd
import numpy as np

In [68]:
import os

In [72]:
name_dataset = "dataset_1"
dataset_path = "datasetsv2/"
length_cut = 10000
random_flag = True

In [70]:
df = pd.read_csv(dataset_path + name_dataset + ".csv")

In [71]:
df.head(5)

,label,text,book
0,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Christine Gehring,...",Right Ho Jeeves
1,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Suzanne L. Shell, ...",Tales Of St Austin
2,Joseph Conrad,And that last\n\nword was the single word of ...,Victory
3,Joseph Conrad,\n\n\n\nIf I have ever had these gifts in any ...,Under Western Eyes
4,Bram Stoker,\n\n\n\n Lond...,The Lady Of The Shroud


In [7]:
from utils.text_processing import get_min_len_corpus

In [8]:
print("Min Length:", get_min_len_corpus(list(df["text"])))

Min Length: 47826


In [9]:
from utils.text_processing import get_corpus, get_random_corpus

In [10]:
texts = list(df['text'])

In [11]:
corpus, segmented_corpus = get_corpus(texts, length_cut)

In [12]:
from utils.text_processing import remove_puntuaction

In [13]:
segmented_corpus = [[[remove_puntuaction(w) for w in t] for t in p] for p in segmented_corpus ]

In [14]:
segmented_corpus[0]

[['Produced',
  'by',
  'Christine',
  'Gehring',
  'Richard',
  'Prairie',
  'and',
  'the',
  'Project',
  'Gutenberg',
  'Online',
  'Distributed',
  'Proofreading',
  'Team',
  'RIGHT',
  'HO',
  'JEEVES',
  'By',
  'P',
  'G',
  'WODEHOUSE',
  '1922',
  'To',
  'RAYMOND',
  'NEEDHAM',
  'KC',
  'WITH',
  'AFFECTION',
  'AND',
  'ADMIRATION',
  '1',
  'Jeeves',
  'I',
  'said',
  'may',
  'I',
  'speak',
  'frankly',
  'Certainly',
  'sir',
  'What',
  'I',
  'have',
  'to',
  'say',
  'may',
  'wound',
  'you',
  'Not',
  'at',
  'all',
  'sir',
  'Well',
  'then',
  'Nowait',
  'Hold',
  'the',
  'line',
  'a',
  'minute',
  'Ive',
  'gone',
  'off',
  'the',
  'rails',
  'I',
  'dont',
  'know',
  'if',
  'you',
  'have',
  'had',
  'the',
  'same',
  'experience',
  'but',
  'the',
  'snag',
  'I',
  'always',
  'come',
  'up',
  'against',
  'when',
  'Im',
  'telling',
  'a',
  'story',
  'is',
  'this',
  'dashed',
  'difficult',
  'problem',
  'of',
  'where',
  'to',
  'be

In [56]:
selected_corpus, words_features, word_index, index_word = get_random_corpus(segmented_corpus,tokenized=True,feature_selection = 'common_words')

['kind', 'of', 'glasstank', 'arrangement', 'and', 'pretty', 'niffy', 'the', 'whole', 'thing', 'was', 'I', 'recall', 'I', 'suppose', 'one', 'ought', 'to', 'have', 'been', 'able', 'to', 'see', 'what', 'the', 'end', 'would', 'be', 'even', 'then', 'but', 'you', 'know', 'what', 'boys', 'are', 'Careless', 'heedless', 'busy', 'about', 'our', 'own', 'affairs', 'we', 'scarcely', 'gave', 'this', 'kink', 'in', 'Gussies', 'character', 'a', 'thought', 'We', 'may', 'have', 'exchanged', 'an', 'occasional', 'remark', 'about', 'it', 'taking', 'all', 'sorts', 'to', 'make', 'a', 'world', 'but', 'nothing', 'more', 'You', 'can', 'guess', 'the', 'sequel', 'The', 'trouble', 'spread', 'Indeed', 'sir', 'Absolutely', 'Jeeves', 'The', 'craving', 'grew', 'upon', 'him', 'The', 'newts', 'got', 'him', 'Arrived', 'at', 'mans', 'estate', 'he', 'retired', 'to', 'the', 'depths', 'of', 'the', 'country', 'and', 'gave', 'his', 'life', 'up', 'to', 'these', 'dumb', 'chums', 'I', 'suppose', 'he', 'used', 'to', 'tell', 'himsel

In [57]:
words_features

{'11': 0,
 '102': 1,
 '215': 2,
 '287': 3,
 '273': 4,
 '153': 5,
 '347': 6,
 '75': 7,
 '79': 8,
 '389': 9}

In [65]:
index_word["153"]

'as'

In [39]:
len(selected_corpus)

78

In [40]:
labels = list(df['label'])

In [41]:
total_classes = list(set(labels))  ## or author
print("Total classes: {}".format(len(total_classes)))
number_books = (df[df['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = {cat: index for index, cat in enumerate(total_classes)}

Total classes: 13
Total entities for each class in train: 6


In [42]:
y = [dict_categories[y] for y in labels]

In [43]:
total_classes

['Joseph Conrad',
 'Pelham Grenville',
 'Allan Poe',
 'Arthur Conan Doyle',
 'Hector Hugh',
 'Thomas Hardy',
 'Charles Darwin',
 'Bram Stoker',
 'Daniel Defoe',
 'George Eliot',
 'Charles Dickens',
 'Mark Twain',
 'Jane Austen']

In [44]:
def graph2vec(operating_system, networks, words_features):
    
    input_path = "struc2vec/graph/"
    output_path = 'struc2vec/emb/'
    try: 
        os.mkdir(input_path)
        os.mkdir(output_path)
    except:
        print("Existe")
    for i, netw in enumerate(networks):
        netw.write_edgelist(input_path+str(i)+".edgelist")
    for i, netw in enumerate(networks):
        if operating_system == 'linux' and i>77:
            path_command = "python struc2vec/src/main.py --input "+input_path+str(i)+".edgelist --output "+output_path+str(i)+".emb"
            #path_command = 'python struc2vec/src/graph2vec.py --input-path ' + in_network + ' --output-path ' + extra_file + ' --dimensions ' + str(512)
            os.system(path_command)
    all_network_features = []        
    for i, netw in enumerate(networks):
        network_features = []
        with open(output_path+str(i)+".emb", "r") as f:
            lines = f.readlines()
            num_token, dim = lines[0].split()
            for l in lines[1:]:
                row = l.strip().split(" ")
                id_node = row[0]
                emb = [float(v) for v in row[1:]]
                if id_node in words_features:
                    network_features.extend(emb)
        network_features = np.array(network_features)
        print(network_features)
        all_network_features.append(network_features)
    return np.array(all_network_features)

In [45]:
def get_graph2vec_features(sequences, words_features):
    network_features = pd.DataFrame()
    nets = []
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=index_word, percentages=None, path="")
        network = obj.create_network()
        nets.append(network)
    X = graph2vec("linux", nets, words_features)
    return X

In [46]:
X = get_graph2vec_features(selected_corpus, words_features)

Nodes: 447 - Edges: 880
Nodes: 467 - Edges: 893
Nodes: 522 - Edges: 941
Nodes: 486 - Edges: 909
Nodes: 450 - Edges: 873
Nodes: 486 - Edges: 909
Nodes: 462 - Edges: 881
Nodes: 461 - Edges: 895
Nodes: 406 - Edges: 856
Nodes: 490 - Edges: 914
Nodes: 457 - Edges: 880
Nodes: 413 - Edges: 878
Nodes: 543 - Edges: 921
Nodes: 368 - Edges: 854
Nodes: 502 - Edges: 928
Nodes: 376 - Edges: 860
Nodes: 519 - Edges: 919
Nodes: 499 - Edges: 912
Nodes: 459 - Edges: 902
Nodes: 457 - Edges: 914
Nodes: 531 - Edges: 917
Nodes: 465 - Edges: 927
Nodes: 430 - Edges: 866
Nodes: 499 - Edges: 913
Nodes: 377 - Edges: 623
Nodes: 458 - Edges: 892
Nodes: 464 - Edges: 895
Nodes: 495 - Edges: 923
Nodes: 469 - Edges: 903
Nodes: 465 - Edges: 874
Nodes: 406 - Edges: 838
Nodes: 494 - Edges: 916
Nodes: 440 - Edges: 841
Nodes: 465 - Edges: 921
Nodes: 320 - Edges: 786
Nodes: 429 - Edges: 878
Nodes: 401 - Edges: 870
Nodes: 448 - Edges: 886
Nodes: 485 - Edges: 892
Nodes: 515 - Edges: 927
Nodes: 444 - Edges: 869
Nodes: 536 - Edg

In [47]:
print("Lenght of features:", X.shape)

Lenght of features: (78, 1024)


# Training with SVM

In [48]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [49]:
RANDOM_SEED = 20
FRAC_TRAIN = 0.8

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=(1.0 - FRAC_TRAIN), random_state=RANDOM_SEED)

In [51]:
X_train.shape

(62, 1024)

In [52]:
X_val.shape

(16, 1024)

In [53]:
from sklearn.svm import SVC

In [54]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(y_val)
print(predicted)
#print accuracy_score
print("Accuracy : " + str(accuracy_score(y_val, predicted)))

print("Micro f-measure " + str(f1_score(y_val, predicted, average='micro')))


[7, 9, 10, 0, 12, 11, 5, 11, 2, 1, 6, 8, 10, 3, 4, 3]
[ 1  2  8  1  7 12  8  0  1  1  1  8  6  8  8 12]
Accuracy : 0.125
Micro f-measure 0.125
